In [1]:
# define matplotlibplotting backend
# %matplotlib -l shows all available backends
%matplotlib qt

In [2]:
import numpy as np
import threading

from lib.parameters import Parameters
from lib.plot_builder import PlotBuilder
from lib.h5_reader import H5Reader
from lib.plain_reader import PlainReader

/home/vyo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
##  configuration options
el_charge = 1.6e-19
rho_beam_scale = 1
config_file = '../testdir/parameters.xml'

cfg = Parameters(config_file)
clim_estimation = cfg.get_clim_estimation()

radius=0.01
longitude=0.1
time_range=[cfg.start_time, cfg.end_time]
use_grid=False
cmap='terrain'

clim_e_r = [-clim_estimation, clim_estimation]
clim_e_z = [-clim_estimation, clim_estimation]
clim_rho_beam = [-(cfg.bunch_density * el_charge * rho_beam_scale), 0]

x_axis_label = r'$\mathit{t (s)}$'
y_r_axis_label = r'$\mathit{E_r (\frac{V}{m})}$'
y_z_axis_label = r'$\mathit{E_z (\frac{V}{m})}$'

e_r_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Radial\enspace Component}\enspace(E_r)$'
e_z_plot_name = r'$\mathbf{Electrical\enspace Field\enspace Longitudal\enspace Component}\enspace(E_z)$'
rho_beam_plot_name = r'$\mathbf{Electron\enspace Bunch\enspace Density}\enspace (\rho_{bunch})$'

In [4]:
# define reader (plain reader used)
if not cfg.use_hdf5:
    reader = PlainReader(cfg.data_path, cfg.system_state_path,
                         [cfg.number_r_grid , cfg.number_z_grid],
                         cfg.frames_per_file, False)
else:
    raise NotImplementedError('HDF5 support still not implemented')

In [5]:
# get data
start_frame = cfg.get_frame_number_by_timestamp(time_range[0])
end_frame = cfg.get_frame_number_by_timestamp(time_range[1])
row_number = cfg.get_row_by_radius(radius)
col_number = cfg.get_col_by_longitude(longitude)

    t4 = threading.Thread(target=task4)
data_r = reader.get_frame_range_dot('E_r', row_number, col_number, start_frame, end_frame - 1)
data_z = reader.get_frame_range_dot('E_z', row_number, col_number, start_frame, end_frame - 1)

0 0 9
22 220 229
0 0 9
22 220 229


In [6]:
the_size = len(data_r)

data_timeline = np.linspace(time_range[0], time_range[1], the_size)

# define plot builder
plot = PlotBuilder(time_range[1], time_range[1],
                   fig_color=cfg.figure_color, 
                   fig_width=cfg.figure_width,
                   fig_height=cfg.figure_height, 
                   fig_dpi=cfg.figure_dpi,
                   font_family=cfg.figure_font_family,
                   font_name=cfg.figure_font_name,
                   font_size=cfg.figure_font_size,
                   tickbox=True, grid=use_grid, is_invert_y_axe=False,
                   aspect='auto', guess_number_ticks=20)

# add subplots
# plot_r = plot.add_subplot_cartesian_2d(e_r_plot_name, 121, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)
# plot_z = plot.add_subplot_cartesian_2d(e_z_plot_name, 122, x_axe_label=x_axis_label, y_axe_label=y_r_axis_label)

# plot.show()

In [7]:
# add data
fig = plot.get_figure()

sp1 = fig.add_subplot(121)
sp2 = fig.add_subplot(122)
sp1.plot(data_timeline, data_r)
sp2.plot(data_timeline, data_z)

plot.show()